In [33]:
spark.stop()

In [1]:
from IPython.core.display import display, HTML, clear_output
display(HTML('<style>.container { width:90% !important; }</style>'))
display(HTML('<style>.prompt { min-width:10ex !important; }</style>'))
display(HTML('<style>div#notebook { font-size:12px !important; }</style>'))

/tmp/ipykernel_18653/4223674425.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML, clear_output
/tmp/ipykernel_18653/4223674425.py:1: DeprecationWarning: Importing clear_output from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML, clear_output


In [2]:
%%html
<style>
div.output_area pre {
    white-space: pre;
}
</style>

## Spark Init

In [3]:
session_name = 'joins'

In [4]:
from mtspark import kinit

kinit(principal='iagalper@MSK.MTS.RU', keytab='/home/iagalper/iagalper.keytab')

In [5]:
from mtspark import get_spark

spark = get_spark({
    "spark.app.name": session_name,
    "spark.driver.memory": "2g",
    "spark.driver.maxResultSize": "2g",
    "spark.executor.cores": "4",
    "spark.executor.memory": "2g",
    "spark.ui.showConsoleProgress": False
}, spark_version="3.5.0")

print('\nsuccess')

:: loading settings :: file = /home/iagalper/adv_venvs/py39-spark3.5/lib/python3.9/site-packages/mtspark/config/ivysettings.xml


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/29 15:49:56 WARN Utils: Service 'sparkDriver' could not bind on port 40000. Attempting port 40001.
24/05/29 15:49:56 WARN Utils: Service 'sparkDriver' could not bind on port 40001. Attempting port 40002.
24/05/29 15:49:56 WARN Utils: Service 'sparkDriver' could not bind on port 40002. Attempting port 40003.
24/05/29 15:49:56 WARN Utils: Service 'sparkDriver' could not bind on port 40003. Attempting port 40004.
24/05/29 15:49:56 WARN Utils: Service 'sparkDriver' could not bind on port 40004. Attempting port 40005.
24/05/29 15:49:56 WARN Utils: Service 'sparkDriver' could not bind on port 40005. Attempting port 40006.
24/05/29 15:49:56 WARN Utils: Service 'sparkDriver' could not bind on port 40006. Attempting port 40007.
24/05/29 15:49:56 WARN Utils: Service 'sparkDriver' could not bind on port 40007. Attempting port 40008.
24/05/29 15:49:56 WARN Utils: 


success


In [6]:
from pyspark.sql import Window
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession, Column
from pyspark.sql import functions as sf

In [7]:
from datetime import datetime, timedelta, date

### tables

In [8]:
df_big = spark.createDataFrame(
    data=[
        ("яблоко", 1, [1,2], datetime.today().date()), 
        ("груша", 2, [3,4], datetime.today().date()),
        ("арбуз", 3, [4,5], datetime.today().date()),
        ("огурец", 4, [6,7], datetime.today().date() - timedelta(1)),
        ("помидор", 5, [8,9], datetime.today().date() - timedelta(1)),
        ("перец", 6, [2,3], datetime.today().date() - timedelta(1)),
        ("редис", 7, [5,6], datetime.today().date() - timedelta(2)),
        ("апельсин", 8, [7,8], datetime.today().date() - timedelta(2)),
        ("лимон", 9, [1,9], datetime.today().date() - timedelta(2)),
        ("тыква", 10, [2,7], datetime.today().date() - timedelta(3))
    ], 
    schema = ["name", "count", "array", "date"]
)

In [9]:
df_big.show(20,0)

+--------+-----+------+----------+
|name    |count|array |date      |
+--------+-----+------+----------+
|яблоко  |1    |[1, 2]|2024-05-29|
|груша   |2    |[3, 4]|2024-05-29|
|арбуз   |3    |[4, 5]|2024-05-29|
|огурец  |4    |[6, 7]|2024-05-28|
|помидор |5    |[8, 9]|2024-05-28|
|перец   |6    |[2, 3]|2024-05-28|
|редис   |7    |[5, 6]|2024-05-27|
|апельсин|8    |[7, 8]|2024-05-27|
|лимон   |9    |[1, 9]|2024-05-27|
|тыква   |10   |[2, 7]|2024-05-26|
+--------+-----+------+----------+



In [10]:
df_big.rdd.getNumPartitions()

200

In [11]:
df_small = spark.createDataFrame(
    data=[
        ("яблоко", 11, [3,4], datetime.today().date()), 
        ("груша", 12, [1,2], datetime.today().date() - timedelta(1)),
        ("арбуз", 13, [1,3], datetime.today().date() - timedelta(2)),
        ("гриб", 14, [4,5], datetime.today().date() - timedelta(3)),
        ("хлеб", 15, [1,8], datetime.today().date() - timedelta(4))
    ], 
    schema = ["name", "count", "array", "date"]
)

In [12]:
df_small.show(20,0)

+------+-----+------+----------+
|name  |count|array |date      |
+------+-----+------+----------+
|яблоко|11   |[3, 4]|2024-05-29|
|груша |12   |[1, 2]|2024-05-28|
|арбуз |13   |[1, 3]|2024-05-27|
|гриб  |14   |[4, 5]|2024-05-26|
|хлеб  |15   |[1, 8]|2024-05-25|
+------+-----+------+----------+



In [13]:
df_big.rdd.getNumPartitions()

200

In [14]:
df_big_1 = df_big.repartition(1)
df_small_1 = df_small.repartition(1)

### equi-join

#### 200 партиций

In [16]:
# применится броадкаст, т.к. есть hint и inner-join

df_big.alias('df_big')\
    .join(
        sf.broadcast(df_small.alias("df_small")), 
        (df_big.name == df_small.name),
        "inner"
    )\
    .explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- BroadcastHashJoin [name#0], [name#25], Inner, BuildRight, false
   :- Filter isnotnull(name#0)
   :  +- Scan ExistingRDD[name#0,count#1L,array#2,date#3]
   +- BroadcastExchange HashedRelationBroadcastMode(List(input[0, string, false]),false), [plan_id=111]
      +- Filter isnotnull(name#25)
         +- Scan ExistingRDD[name#25,count#26L,array#27,date#28]




In [17]:
# применится броадкаст, т.к. есть hint и lef-join

df_big.alias('df_big')\
    .join(
        sf.broadcast(df_small.alias("df_small")), 
        (df_big.name == df_small.name),
        "left"
    )\
    .explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- BroadcastHashJoin [name#0], [name#25], LeftOuter, BuildRight, false
   :- Scan ExistingRDD[name#0,count#1L,array#2,date#3]
   +- BroadcastExchange HashedRelationBroadcastMode(List(input[0, string, false]),false), [plan_id=129]
      +- Filter isnotnull(name#25)
         +- Scan ExistingRDD[name#25,count#26L,array#27,date#28]




In [18]:
# здесь будет sort merge, несмотря на hint, т.к. используем full-join

df_big.alias('df_big')\
    .join(
        sf.broadcast(df_small.alias("df_small")), 
        (df_big.name == df_small.name),
        "full"
    )\
    .explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- SortMergeJoin [name#0], [name#25], FullOuter
   :- Sort [name#0 ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(name#0, 200), ENSURE_REQUIREMENTS, [plan_id=144]
   :     +- Scan ExistingRDD[name#0,count#1L,array#2,date#3]
   +- Sort [name#25 ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(name#25, 200), ENSURE_REQUIREMENTS, [plan_id=145]
         +- Scan ExistingRDD[name#25,count#26L,array#27,date#28]




In [19]:
# форсим применение shuffle hash, можно не выставлять spark.sql.join.preferSortMergeJoin в false, т.к. у нас есть hint

df_big.alias('df_big')\
    .join(
        df_small.alias("df_small").hint('SHUFFLE_HASH'), 
        (df_big.name == df_small.name),
        "inner"
    )\
    .explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- ShuffledHashJoin [name#0], [name#25], Inner, BuildRight
   :- Exchange hashpartitioning(name#0, 200), ENSURE_REQUIREMENTS, [plan_id=170]
   :  +- Filter isnotnull(name#0)
   :     +- Scan ExistingRDD[name#0,count#1L,array#2,date#3]
   +- Exchange hashpartitioning(name#25, 200), ENSURE_REQUIREMENTS, [plan_id=171]
      +- Filter isnotnull(name#25)
         +- Scan ExistingRDD[name#25,count#26L,array#27,date#28]




In [20]:
# shuffle hash работает и с full-join

df_big.alias('df_big')\
    .join(
        df_small.alias("df_small").hint('SHUFFLE_HASH'), 
        (df_big.name == df_small.name),
        "full"
    )\
    .explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- ShuffledHashJoin [name#0], [name#25], FullOuter, BuildRight
   :- Exchange hashpartitioning(name#0, 200), ENSURE_REQUIREMENTS, [plan_id=186]
   :  +- Scan ExistingRDD[name#0,count#1L,array#2,date#3]
   +- Exchange hashpartitioning(name#25, 200), ENSURE_REQUIREMENTS, [plan_id=187]
      +- Scan ExistingRDD[name#25,count#26L,array#27,date#28]




In [21]:
# а тут Sort Merge, т.к. нет hint

df_big.alias('df_big')\
    .join(
        df_small.alias("df_small"), 
        (df_big.name == df_small.name),
        "full"
    )\
    .explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- SortMergeJoin [name#0], [name#25], FullOuter
   :- Sort [name#0 ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(name#0, 200), ENSURE_REQUIREMENTS, [plan_id=202]
   :     +- Scan ExistingRDD[name#0,count#1L,array#2,date#3]
   +- Sort [name#25 ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(name#25, 200), ENSURE_REQUIREMENTS, [plan_id=203]
         +- Scan ExistingRDD[name#25,count#26L,array#27,date#28]




#### 1 партиция

In [22]:
# применится броадкаст, т.к. анализатор понял, что таблички маленькие

df_big_1.alias('df_big_1')\
    .join(
        df_small_1.alias("df_small_1"), 
        (df_big_1.name == df_small_1.name),
        "inner"
    )\
    .explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- SortMergeJoin [name#0], [name#25], Inner
   :- Sort [name#0 ASC NULLS FIRST], false, 0
   :  +- Exchange SinglePartition, REPARTITION_BY_NUM, [plan_id=229]
   :     +- Filter isnotnull(name#0)
   :        +- Scan ExistingRDD[name#0,count#1L,array#2,date#3]
   +- Sort [name#25 ASC NULLS FIRST], false, 0
      +- Exchange SinglePartition, REPARTITION_BY_NUM, [plan_id=232]
         +- Filter isnotnull(name#25)
            +- Scan ExistingRDD[name#25,count#26L,array#27,date#28]




### non equi-join

#### 200 партиций

In [23]:
# cartesian product, т.к. нет ни одного ключа на равенство

df_big.alias('df_big')\
    .join(
        df_small.alias("df_small"), 
        (df_big.name != df_small.name)
    )\
    .explain()

== Physical Plan ==
CartesianProduct NOT (name#0 = name#25)
:- *(1) Filter isnotnull(name#0)
:  +- *(1) Scan ExistingRDD[name#0,count#1L,array#2,date#3]
+- *(2) Filter isnotnull(name#25)
   +- *(2) Scan ExistingRDD[name#25,count#26L,array#27,date#28]




In [24]:
# brodcast nested loop, т.к. cartesian product не поддерживает outer джойны

df_big.alias('df_big')\
    .join(
        df_small.alias("df_small"), 
        (df_big.name != df_small.name), 
        'full'
    )\
    .explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- BroadcastNestedLoopJoin BuildRight, FullOuter, NOT (name#0 = name#25)
   :- Scan ExistingRDD[name#0,count#1L,array#2,date#3]
   +- BroadcastExchange IdentityBroadcastMode, [plan_id=273]
      +- Scan ExistingRDD[name#25,count#26L,array#27,date#28]




In [25]:
# sort merge, т.к. есть одно условие на равенство ключей
df_big.alias('df_big')\
    .join(
        df_small.alias("df_small"), 
        (
            (df_big.name == df_small.name) & (df_big.date != df_small.date)
        ),
        'full'
    )\
    .explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- SortMergeJoin [name#0], [name#25], FullOuter, NOT (date#3 = date#28)
   :- Sort [name#0 ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(name#0, 200), ENSURE_REQUIREMENTS, [plan_id=288]
   :     +- Scan ExistingRDD[name#0,count#1L,array#2,date#3]
   +- Sort [name#25 ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(name#25, 200), ENSURE_REQUIREMENTS, [plan_id=289]
         +- Scan ExistingRDD[name#25,count#26L,array#27,date#28]




In [28]:
# cartesian product, т.к. нет ни одного ключа на равенство

df_big.alias('df_big')\
    .join(
        df_small.alias("df_small"), 
        (
            sf.size(sf.array_except(df_big.array, df_small.array)) > 0
        )
    )\
    .explain()

== Physical Plan ==
CartesianProduct (size(array_except(array#2, array#27), true) > 0)
:- *(1) Scan ExistingRDD[name#0,count#1L,array#2,date#3]
+- *(2) Scan ExistingRDD[name#25,count#26L,array#27,date#28]




In [31]:
# brodcast nested loop, т.к. нет ни одного ключа на равенство и есть hint на broadcast

df_big.alias('df_big')\
    .join(
        sf.broadcast(df_small.alias("df_small")), 
        (
            sf.size(sf.array_except(df_big.array, df_small.array)) > 0
        ),
        'full'
    )\
    .explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- BroadcastNestedLoopJoin BuildRight, FullOuter, (size(array_except(array#2, array#27), true) > 0)
   :- Scan ExistingRDD[name#0,count#1L,array#2,date#3]
   +- BroadcastExchange IdentityBroadcastMode, [plan_id=319]
      +- Scan ExistingRDD[name#25,count#26L,array#27,date#28]




In [32]:
# sort merge, т.к. как минимум с 3 версии спарка, массивы сортируемы

df_big.alias('df_big')\
    .join(
        df_small.alias("df_small"), 
        (
            (df_big.array == df_small.array)
        )
    )\
    .explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- SortMergeJoin [array#2], [array#27], Inner
   :- Sort [array#2 ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(array#2, 200), ENSURE_REQUIREMENTS, [plan_id=342]
   :     +- Filter isnotnull(array#2)
   :        +- Scan ExistingRDD[name#0,count#1L,array#2,date#3]
   +- Sort [array#27 ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(array#27, 200), ENSURE_REQUIREMENTS, [plan_id=343]
         +- Filter isnotnull(array#27)
            +- Scan ExistingRDD[name#25,count#26L,array#27,date#28]




#### 1 партиция

In [ ]:
# brodcast nested loop, т.к. анализатор понял, что таблички маленькие

df_big_1.alias('df_big_1')\
    .join(
        df_small_1.alias("df_small_1"), 
        (
            F.size(F.array_except(df_big_1.array, df_small_1.array)) > 0
        ),
        'full'
    )\
    .show()

In [ ]:
spark.stop()